İnfoDataset


https://docs.google.com/spreadsheets/d/1WfTdd1O3WbmHf6oIrvoVBdDcmDt4VlUyh28827OYZnI/edit?gid=1610021778#gid=1610021778

1. Veri Yükleme ve Hazırlık

In [ ]:
import pandas as pd

# Büyük dosya olan 'order_products__prior.csv' yüklenirken biraz zaman alabilir.

print("--- Tüm Instacart CSV Dosyaları Yükleniyor ---")

# 1. Koridorlar (Aisles)
df_aisles = pd.read_csv('aisles.csv')
print(f"1. df_aisles: {len(df_aisles)} satır yüklendi.")

# 2. Departmanlar (Departments)
df_departments = pd.read_csv('departments.csv')
print(f"2. df_departments: {len(df_departments)} satır yüklendi.")

# 3. Ürünler (Products)
df_products = pd.read_csv('products.csv')
print(f"3. df_products: {len(df_products)} satır yüklendi.")

# 4. Sipariş Detayları (Orders)
df_orders = pd.read_csv('orders.csv')
print(f"4. df_orders: {len(df_orders)} satır yüklendi.")

# 5. Önceki Sipariş Ürünleri (Prior Orders - EN BÜYÜK DOSYA)
df_op_prior = pd.read_csv('order_products__prior.csv')
print(f"5. df_op_prior: {len(df_op_prior)} satır yüklendi.")

# 6. Eğitim Sipariş Ürünleri (Train Orders)
df_op_train = pd.read_csv('order_products__train.csv')
print(f"6. df_op_train: {len(df_op_train)} satır yüklendi.")

print("\n🎉 Tüm 6 DataFrame başarıyla yüklendi ve kullanıma hazır!")

2. Genel Yapısal İnceleme

In [ ]:
## 📏 Boyut ve Şekil
print("Veri Boyutu (Satır, Sütun):", df.shape)
print("-" * 30)

## 👁️ İlk Gözlem
print("İlk 5 Satır:")
print(df.head())
print("-" * 30)

## ℹ️ Sütun Bilgileri ve Eksik Değer Kontrolü
print("Sütun Tipleri ve Eksik Olmayan Değerler:")
df.info()
print("-" * 30)

## 💡 İstatistiksel Özet
# Sadece sayısal sütunlar için:
print("Sayısal Sütunların İstatistiksel Özeti:")
print(df.describe())

# Kategorik sütunları da dahil etmek için:
print("Tüm Sütunların İstatistiksel Özeti (Kategorik dahil):")
print(df.describe(include='all'))

In [ ]:
## 🗑️ Eksik Değerlerin Toplamı
print("Eksik Değerlerin Sütun Bazında Toplamı:")
print(df.isnull().sum())
print("-" * 30)

## 📊 Eksik Değerlerin Yüzdesi
print("Eksik Değerlerin Yüzdesi:")
print((df.isnull().sum() / len(df)) * 100)
print("-" * 30)

## 🔢 Benzersiz Değer Sayısı
print("Benzersiz Değer Sayısı:")
print(df.nunique())

In [ ]:
# Örnek: 'department' sütununu inceleme (products.csv dosyasında mevcut)
kategorik_sutun = 'department_id'

## ⚖️ Değer Sıklığı
print(f"'{kategorik_sutun}' Sütunu Değer Sıklıkları:")
print(df[kategorik_sutun].value_counts())
print("-" * 30)

## 🔝 En Sık Görülen Değerler
print(f"'{kategorik_sutun}' Sütunu En Sık Görülen 10 Değer:")
print(df[kategorik_sutun].value_counts().head(10))

3. 📅 Sipariş Zamanlaması Analizi

---



In [ ]:
print("--- DF_ORDERS YAPISAL ANALİZ ---")
print(df_orders.info())
print("\nSayısal Özet:")
print(df_orders.describe())

In [ ]:
plt.figure(figsize=(14, 5))

# 1. Siparişlerin Haftanın Gününe Göre Dağılımı (order_dow)
plt.subplot(1, 2, 1)
sns.countplot(x='order_dow', data=df_orders)
plt.title('Siparişlerin Haftanın Gününe Göre Dağılımı (0=Paz)')
plt.xlabel('Haftanın Günü')
plt.ylabel('Sipariş Sayısı')

# 2. Siparişlerin Günün Saatlerine Göre Dağılımı (order_hour_of_day)
plt.subplot(1, 2, 2)
sns.countplot(x='order_hour_of_day', data=df_orders)
plt.title('Siparişlerin Günün Saatlerine Göre Dağılımı')
plt.xlabel('Gün Saati')
plt.ylabel('Sipariş Sayısı')

plt.tight_layout()
plt.show()

4. 🛍️ Ürün ve Sepet Analizi

In [ ]:
print("--- DF_OP_PRIOR YAPISAL ANALİZ ---")
# Bu dosya çok büyük olduğundan sadece ilk 5 satıra ve info'ya bakmak önemlidir
print(df_op_prior.head())
print("\nSütun Tipleri:")
print(df_op_prior.dtypes)

In [ ]:
plt.figure(figsize=(6, 5))

# 1. Yeniden Sipariş Oranı
reorder_rate = df_op_prior['reordered'].value_counts(normalize=True) * 100

sns.barplot(x=reorder_rate.index, y=reorder_rate.values)
plt.title('Ürünlerin Yeniden Sipariş Edilme Oranı')
plt.xlabel('Yeniden Sipariş Edildi (0=Hayır, 1=Evet)')
plt.ylabel('Yüzde (%)')
plt.xticks([0, 1], ['Hayır', 'Evet'])
plt.show()

Yorum: Instacart'ta satılan ürünlerin büyük bir kısmı (%59 civarı) geçmişte zaten sipariş edilmiş ürünlerdir. Bu, sadık müşteri davranışını ve tahmin modellerinin bu eğilimden faydalanma potansiyelini gösterir.

In [ ]:
# Sadece ilgili kolonları seçerek bellek kullanımını azaltma
aisles = df_aisles[['aisle_id', 'aisle']]
products = df_products[['product_id', 'aisle_id']]

# 1. Prior Orders ile Products birleştirme (hangi siparişte hangi ürün satıldı?)
merged_df = pd.merge(df_op_prior, products, on='product_id', how='left')

# 2. Sonuç ile Aisles birleştirme (ürün hangi koridordan?)
final_df = pd.merge(merged_df, aisles, on='aisle_id', how='left')

# En popüler 10 koridoru bulma
top_aisles = final_df['aisle'].value_counts().nlargest(10)

plt.figure(figsize=(10, 7))
sns.barplot(x=top_aisles.values, y=top_aisles.index)
plt.title('En Çok Sipariş Edilen 10 Koridor (Aisle)')
plt.xlabel('Satılan Ürün Sayısı')
plt.ylabel('Koridor Adı')
plt.show()

In [ ]:
import pandas as pd
# df_products'ın daha önce yüklendiğini varsayalım
# df_products = pd.read_csv('products.csv')

print("--- DF_PRODUCTS YAPISAL ANALİZ ---")
print(df_products.info())
print("\nEksik Değerler:")
print(df_products.isnull().sum())
print("-" * 30)

In [ ]:
# df_products'ın benzersiz değerlerini hesaplayalım
print(f"Toplam Ürün Kayıt Sayısı (Satır): {len(df_products)}")
print(f"Benzersiz Ürün Kimliği (product_id) Sayısı: {df_products['product_id'].nunique()}")
print(f"Benzersiz Ürün Adı (product_name) Sayısı: {df_products['product_name'].nunique()}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 5))

# 1. Departman (department_id) Dağılımı
plt.subplot(1, 2, 1)
# Değerleri sayısal değil, string olarak kabul edip sıralama yapmadan görselleştirme
df_products['department_id'].value_counts().plot(kind='bar')
plt.title('Ürünlerin Departman ID Bazında Çeşitliliği')
plt.xlabel('Departman ID')
plt.ylabel('Ürün Çeşidi Sayısı')

# 2. Koridor (aisle_id) Dağılımı
plt.subplot(1, 2, 2)
df_products['aisle_id'].value_counts().nlargest(10).plot(kind='bar')
plt.title('En Çeşitli 10 Koridor ID')
plt.xlabel('Koridor ID')
plt.ylabel('Ürün Çeşidi Sayısı')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



# 1. Products ile Departments birleştirme (department_id üzerinden)
df_product_catalog = pd.merge(df_products, df_departments[['department_id', 'department']], on='department_id', how='left')

# 2. Üzerine Aisles birleştirme (aisle_id üzerinden)
df_product_catalog = pd.merge(df_product_catalog, df_aisles[['aisle_id', 'aisle']], on='aisle_id', how='left')

print("--- Birleştirilmiş Ürün Kataloğu İlk 5 Satır ---")
print(df_product_catalog.head())
print("-" * 30)


# 3. Görselleştirme: Departmanlara Göre Ürün Çeşitliliği (İsimlerle)
top_10_departments_name = df_product_catalog['department'].value_counts().nlargest(10)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_departments_name.values, y=top_10_departments_name.index)
plt.title('Departmanlara Göre Ürün Çeşitliliği (Top 10)')
plt.xlabel('Ürün Çeşidi Sayısı')
plt.ylabel('Departman Adı')
plt.savefig('departman_cesitliligi_isimli.png')
plt.close()

print("Kategori hiyerarşisi (isimli) görselleştirildi.")

Evet (Vekil Kullanılarak) Instacart veri setinde ürün fiyatları bulunmadığı için,  ürünlerin Departmanına (Department) göre bir fiyat aralığı belirleyip, random fonksiyonunu bu aralıklar içinde kullanarak Monetary için gereken veri oluşturulmuştur.

In [ ]:
import numpy as np
import pandas as pd

# 1. Ürünler ve Departmanları Birleştir (Eğer daha önce yapılmadıysa)
# df_products ve df_departments dataframe'lerinizin yüklü olduğunu varsayıyorum
df_products_w_dept = pd.merge(df_products, df_departments, on='department_id', how='left')

# 2. Departmanlara Göre Tahmini Fiyat Aralıkları (Min, Max $) Tanımlama
# Bu değerler market ortalamalarına göre tahmimidir.
dept_pricing = {
    'produce': (1, 5),        # Meyve-Sebze: Ucuz
    'dairy eggs': (2, 8),     # Süt-Yumurta: Orta
    'snacks': (2, 6),         # Atıştırmalık: Ucuz-Orta
    'beverages': (1, 8),      # İçecek: Değişken
    'frozen': (3, 12),        # Dondurulmuş: Orta-Pahalı
    'pantry': (2, 10),        # Kiler: Orta
    'bakery': (3, 9),         # Fırın: Orta
    'canned goods': (1, 5),   # Konserve: Ucuz
    'deli': (5, 15),          # Şarküteri: Pahalı
    'dry goods pasta': (2, 8),# Kuru Gıda: Orta
    'household': (5, 20),     # Ev Gereçleri: Pahalı
    'meat seafood': (10, 30), # Et/Balık: En Pahalı
    'personal care': (4, 15), # Kişisel Bakım: Orta-Pahalı
    'babies': (10, 25),       # Bebek: Pahalı
    'alcohol': (10, 40),      # Alkol: En Pahalı
    'pets': (5, 20),          # Evcil Hayvan: Pahalı
    'breakfast': (3, 8),      # Kahvaltılık: Orta
    'international': (4, 12), # İthal Ürünler: Orta-Pahalı
    'missing': (5, 5),        # Bilinmeyen: Sabit
    'other': (5, 5),
    'bulk': (2, 10)
}

# 3. Rastgele Fiyat Atama Fonksiyonu
def assign_price(dept_name):
    # Sözlükte varsa aralığı al, yoksa varsayılan (3, 10) kullan
    low, high = dept_pricing.get(dept_name, (3, 10))
    # O aralıkta rastgele bir float sayı üret ve yuvarla
    return round(np.random.uniform(low, high), 2)

# 4. Fiyatları Ürünlere Uygulama
# seed ekleyelim ki her çalıştırdığınızda fiyatlar değişmesin
np.random.seed(42)

df_products_w_dept['estimated_price'] = df_products_w_dept['department'].apply(assign_price)

print("Ürünlere departman bazlı mantıklı fiyatlar atandı:")
print(df_products_w_dept[['product_name', 'department', 'estimated_price']].head(10))

In [ ]:
# 1. Önceki sipariş detaylarını (df_op_prior) al
# Sadece ürün ID ve Sipariş ID lazım
df_items = df_op_prior[['order_id', 'product_id']]

# 2. Fiyat bilgisini buraya ekle (product_id üzerinden merge)
df_items = pd.merge(df_items, df_products_w_dept[['product_id', 'estimated_price']], on='product_id', how='left')

# 3. Sipariş başına Toplam Tutar Hesapla (Burası artık sepet büyüklüğü değil, dolar tutarıdır)
order_monetary = df_items.groupby('order_id')['estimated_price'].sum().reset_index()
order_monetary.columns = ['order_id', 'total_spent']

# 4. Ana Sipariş Tablosu (df_orders) ile Birleştirme
df_rfm_base = df_orders[df_orders['eval_set'] == 'prior'].copy()
df_rfm_base = pd.merge(df_rfm_base, order_monetary, on='order_id', how='left')
df_rfm_base['total_spent'] = df_rfm_base['total_spent'].fillna(0)

# 5. YENİ RFM HESAPLAMASI (Monetary artık gerçekçi bir para değeri)
rfm = df_rfm_base.groupby('user_id').agg({
    'days_since_prior_order': lambda x: x.iloc[-1],  # Recency
    'order_number': 'max',                           # Frequency
    'total_spent': 'sum'                             # Monetary (Toplam Harcanan Para)
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm['Recency'] = rfm['Recency'].fillna(0)

print("YENİ (Fiyat Tahminli) RFM Metrikleri:")
print(rfm.head())

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Önce Skorları Tekrar Hesaplayalım (Çünkü tablo yenilendiği için bunlar da silinmiş olabilir)
# Not: qcut hata verirse (aynı değerler çoksa) rank(method='first') kullanımı hayat kurtarır.
rfm["Recency_Score"] = pd.qcut(rfm['Recency'].rank(method="first"), 5, labels=[5, 4, 3, 2, 1])
rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

# 2. RFM Skorunu Metin Olarak Birleştirme
rfm["RFM_SCORE"] = (rfm['Recency_Score'].astype(str) +
                    rfm['Frequency_Score'].astype(str))

# 3. Segment Haritasını Tekrar Tanımlayıp Uygulama
seg_map = {
    r'[1-2][1-2]': 'Hibernating (Uykuda)',
    r'[1-2][3-4]': 'At_Risk (Risk Altında)',
    r'[1-2]5': 'Can\'t_Loose (Kaybedilemez)',
    r'3[1-2]': 'About_to_Sleep (Uykuya Dalmak Üzere)',
    r'33': 'Need_Attention (Dikkat Gerektirir)',
    r'[3-4][4-5]': 'Loyal_Customers (Sadık Müşteriler)',
    r'41': 'Promising (Umut Vaat Eden)',
    r'51': 'New_Customers (Yeni Müşteriler)',
    r'[4-5][2-3]': 'Potential_Loyalists (Potansiyel Sadık)',
    r'5[4-5]': 'Champions (Şampiyonlar)'
}

rfm['Segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

print("Segmentler başarıyla yeniden oluşturuldu.")
print(rfm[['Recency', 'Frequency', 'Monetary', 'Segment']].head())

# 4. Şimdi Grafiği Çizdirelim
plt.figure(figsize=(16, 8))

sns.scatterplot(
    data=rfm,
    x='Recency',
    y='Frequency',
    hue='Segment',
    palette='tab10',
    s=50,
    alpha=0.6
)

plt.title('Müşteri Segmentleri: Recency vs Frequency', fontsize=15)
plt.xlabel('Recency (Son Siparişten Geçen Gün)', fontsize=12)
plt.ylabel('Frequency (Toplam Sipariş Sayısı)', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

Betimsel Analiz: Terk edenler ile kalanlar arasındaki farklar neler? (Sayısal Karşılaştırma)
Makine Öğrenmesi (Machine Learning): Hangi faktörler müşterinin terk etmesine sebep oluyor? (Random Forest Modeli)
Instacart verisinde "Terk Etme" (Churn) tanımımız: Son 30 gündür sipariş vermeyenler (days_since_prior_order == 30).
Bu kodları sırasıyla çalıştırın.
Adım 1: Churn Etiketleme ve İstatistiksel Karşılaştırma
Önce "Aktif" ve "Churn" olanları etiketleyip, davranış farklarına bakalım. Terk edenler daha mı az harcıyordu? Daha mı az ürün alıyordu?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Churn Etiketi Oluşturma (Eğer önceki adımlarda silindiyse)
# Recency değeri 30 olanlar (veri setindeki tavan değer) bizim için Churn'dür.
rfm['Churn_Flag'] = rfm['Recency'].apply(lambda x: 1 if x >= 30 else 0)

# 2. Ortalamaların Karşılaştırılması
churn_summary = rfm.groupby('Churn_Flag').agg({
    'Frequency': 'mean',
    'Monetary': 'mean',          # Toplam Harcama
    'Recency': 'mean',           # Gün farkı (Doğal olarak Churn grubunda yüksek çıkacak)
    'Churn_Flag': 'count'        # Kişi Sayısı
}).rename(columns={'Churn_Flag': 'Count'})

# İsimlendirme
churn_summary.index = ['Aktif (0)', 'Churn / Terk (1)']

print("--- AKTİF vs CHURN MÜŞTERİ KARŞILAŞTIRMASI ---")
print(churn_summary)
print("-" * 50)

# 3. Sepet Başına Ortalama Harcama (Average Ticket Size)
# Bunu türetmek, müşterinin "pinti" mi yoksa "bonkör" mü olduğunu anlamamızı sağlar.
rfm['Avg_Order_Value'] = rfm['Monetary'] / rfm['Frequency']

print("\nSepet Başına Ortalama Harcama Farkı:")
print(rfm.groupby('Churn_Flag')['Avg_Order_Value'].mean())

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 1. Toplam Sipariş Sayısı (Frequency)
sns.boxplot(ax=axes[0], x='Churn_Flag', y='Frequency', data=rfm, showfliers=False, palette="Set2")
axes[0].set_title('Sipariş Sıklığı (Frequency)')
axes[0].set_xticklabels(['Aktif', 'Terk'])

# 2. Toplam Harcama (Monetary)
sns.boxplot(ax=axes[1], x='Churn_Flag', y='Monetary', data=rfm, showfliers=False, palette="Set2")
axes[1].set_title('Toplam Harcama (Monetary)')
axes[1].set_xticklabels(['Aktif', 'Terk'])

# 3. Sepet Başına Ortalama Değer
sns.boxplot(ax=axes[2], x='Churn_Flag', y='Avg_Order_Value', data=rfm, showfliers=False, palette="Set2")
axes[2].set_title('Ortalama Sepet Tutarı ($)')
axes[2].set_xticklabels(['Aktif', 'Terk'])

plt.suptitle('Churn Analizi: Müşteri Davranış Dağılımları', fontsize=16)
plt.tight_layout()
plt.show()

Adım 3: Churn Tahminlemesi (Makine Öğrenmesi - Random Forest)
Şimdi işi ileri seviyeye taşıyalım. Basit bir Yapay Zeka modeli kurarak "Bir müşterinin terk etmesini en çok etkileyen faktör nedir?" sorusuna cevap arayalım.
Not: Recency sütununu modele sokmayacağız çünkü Churn tanımımız zaten Recency üzerine kurulu (Data Leakage olmaması için).

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. Model İçin Veri Hazırlığı
# Girdi (X): Frequency, Monetary, Avg_Order_Value (Recency ÇIKARILDI!)
X = rfm[['Frequency', 'Monetary', 'Avg_Order_Value']]
# Hedef (y): Churn_Flag (1 veya 0)
y = rfm['Churn_Flag']

# Eğitim ve Test setine ayırma (%70 eğitim, %30 test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Modeli Kurma (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 3. Tahmin Yapma
y_pred = rf_model.predict(X_test)

# 4. Sonuçları Değerlendirme
print("--- MODEL PERFORMANSI ---")
print("Doğruluk (Accuracy):", accuracy_score(y_test, y_pred))
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))

# 5. Özellik Önem Düzeyleri (Feature Importance)
# Hangi faktör churn üzerinde en etkili?
import numpy as np

feature_imp = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(x=feature_imp, y=feature_imp.index, palette='viridis')
plt.title("Müşteri Terkini (Churn) Etkileyen En Önemli Faktörler")
plt.xlabel("Önem Düzeyi Skoru")
plt.show()

1. ⚖️ Dengesizlik Giderme (Balancing)
Modelin çoğunluk sınıfına kaymasını engellemek için sınıf ağırlıklandırması uygulayalım. Bu, Random Forest'ın yerleşik bir özelliğidir ve azınlık sınıfı örneklerine daha fazla önem verir.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Sınıf ağırlığını 'balanced' (dengelenmiş) olarak ayarlama.
# Bu, modelin azınlık sınıfı (Sınıf 1) hatalarını daha çok cezalandırmasını sağlar.
rf_model_balanced = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'  # 👈 Ana Değişiklik
)

# Modeli eğitme
rf_model_balanced.fit(X_train, y_train)

# Tahmin yapma ve Değerlendirme
y_pred_balanced = rf_model_balanced.predict(X_test)

print("--- 1. Sınıf Ağırlıklandırması ile MODEL PERFORMANSI ---")
print(classification_report(y_test, y_pred_balanced))

SMOTE
Eğer ağırlıklandırma yeterli gelmezse, eğitim verisini yapay olarak çoğaltan (oversampling) SMOTE tekniğini kullanabilirsiniz (imblearn kütüphanesi gereklidir):

In [ ]:
 from imblearn.over_sampling import SMOTE
 from imblearn.pipeline import Pipeline

# # SMOTE ile eğitim verisini çoğaltma ve Random Forest'ı birleştirme
 pipeline = Pipeline([
     ('smote', SMOTE(random_state=42)),
     ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
 ])

 pipeline.fit(X_train, y_train)
 y_pred_smote = pipeline.predict(X_test)
 print(classification_report(y_test, y_pred_smote))

🧠 Özellik Mühendisliği (Feature Engineering)
Modelin tahmin yeteneğini artırmak için yeni, ayırt edici özellikler (Recency veya Churn_Flag'in nedenini açıklayan) ekleyelim.

Aşağıdaki kod, Ortalama Ürün Çeşitliliği (müşterinin tek bir siparişte kaç farklı ürün aldığı) adında güçlü bir özellik eklemeyi gösterir. Bu, mevcut rfm tablosunun nasıl zenginleştirileceğine dair bir örnektir:

In [ ]:
# Varsayım: df_op_prior ve df_orders yüklü ve birleştirilmiş bir DataFrame'iniz (df_merged) var.

# 1. Yeni Özellik Hesaplama: Ortalama Ürün Çeşitliliği (Average Product Variety)
# Her bir siparişteki benzersiz ürün sayısını bulur.
product_variety = df_op_prior.groupby('order_id')['product_id'].nunique().reset_index(name='unique_products_in_order')

# Her user_id'nin ortalama sipariş çeşitliliğini hesaplar ve rfm tablosuna ekler.
avg_variety_per_user = pd.merge(df_orders[['order_id', 'user_id']], product_variety, on='order_id', how='left')
avg_variety_per_user = avg_variety_per_user.groupby('user_id')['unique_products_in_order'].mean().reset_index(name='Avg_Variety')

# Mevcut RFM tablosuna ekleme
rfm = pd.merge(rfm, avg_variety_per_user, on='user_id', how='left')

# KOD ÖRNEĞİ: Yeni X veri setini oluşturma ve yeniden eğitme
X_new = rfm[['Frequency', 'Monetary', 'Avg_Order_Value', 'Avg_Variety']] # 👈 Yeni Özellik eklendi

# Modelin yeniden eğitilmesi
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)
rf_model_new_features = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_new_features.fit(X_train, y_train)
# ... Değerlendirme ...

GridSearchCV tüm parametre kombinasyonlarını deneyecek ve azınlık sınıfının Recall (yakalama) oranını en çok artıran hiperparametre setini size döndürecektir.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score
from sklearn.ensemble import RandomForestClassifier  # Bu da gereklidir!

# Varsayım: X_train, y_train, X_test, y_test değişkenleri daha önce tanımlanmıştır.

# 1. Metrik Tanımlama: Odak noktamızı Sınıf 1 (Pozitif Sınıf) Recall olarak belirleyelim.
recall_scorer = make_scorer(recall_score, pos_label=1)

# 2. Aranacak Parametreler
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 8, None],
    'min_samples_split': [2, 5],
    'class_weight': ['balanced']
}

# 3. GridSearchCV Kurulumu
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring=recall_scorer,  # Recall'ı optimize et!
    cv=3,
    n_jobs=-1 # Tüm CPU çekirdeklerini kullan
)

# 4. Arama İşlemini Çalıştırma
grid_search.fit(X_train, y_train) # 👈 Modeli eğitim verisiyle eğitme

# 5. En İyi Sonuçları Görüntüleme
print("--- GRID SEARCH SONUÇLARI ---")
print("En İyi Parametreler:", grid_search.best_params_)
print("En İyi Recall Skoru (Çapraz Doğrulama):", grid_search.best_score_)

# 6. En İyi Modeli Kullanarak Test Setinde Değerlendirme
best_rf_model = grid_search.best_estimator_
y_pred_tuned = best_rf_model.predict(X_test)

from sklearn.metrics import classification_report
print("\nEn İyi Model ile Test Seti Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_tuned))